<a href="https://colab.research.google.com/github/johntango/PS2HouseDataExercise/blob/master/Predict_House_Prices_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# House Sales Data, ETL, Build Predictive Model(linear regression), 
# Assess Model, Deploy Model   'houseSmallData.csv' and "data_description.txt"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### data_description.txt
SalePrice — the property’s sale price in dollars. This is the target variable that you’re trying to predict.  
MSSubClass — The building class  
MSZoning — The general zoning classification 
MasVnrArea - Masonary Veneer Area in sq ft
LotFrontage — Linear feet of street connected to property  
LotArea — Lot size in square feet  
Street — Type of road access  
Alley — Type of alley access  
LotShape — General shape of property  
LandContour — Flatness of the property  
Utilities — Type of utilities available  
LotConfig — Lot configuration  

$$Y = X_{1}\beta_{1} + X_{2}\beta_{2} + X_{3}\beta_{3}...$$

In [2]:
data = pd.read_csv('train.csv')
# show all columns
pd.set_option('display.max_columns', None)
# print(train.head())


data.to_json('housedata.json')
print(data.shape)
data.head()



(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# define a function to encode the categorical variables
from sklearn import preprocessing
def encode_text_index(df):
    result = df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

In [4]:
# define a function to normalize the data
def normalize_numeric_minmax(df):
    result = df.copy()
    for column in result.columns:
        # skip the target column
        if column == 'SalePrice':
            continue
        # skip the id column
        if column == 'Id':
            continue
        if result.dtypes[column] == np.int64 or result.dtypes[column] == np.float64:
            max_value = result[column].max()
            min_value = result[column].min()
            result[column] = (result[column] - min_value) / (max_value - min_value)
    return result

In [5]:
# define a function to apply l2 linear regression model to the data
from sklearn.linear_model import Ridge
def linear_regression(x, y):
    model = Ridge(alpha=0.1)
    model.fit(x, y)
    return model

In [6]:
# define a function to calculate the root mean squared error
from sklearn.metrics import mean_squared_error
def rmse(model, x_test, y_test):
    y_pred = model.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))


In [7]:
# check nan values in the data
for cols in data.columns:
# drop the columns with nan values
    if data[cols].isna().sum() > 0:
        data = data.drop(cols, axis=1)

In [8]:
# start with all features included
# remove one feature each time after training the model and compare the validate MSE
# if the MSE is lower, keep the feature, otherwise remove it
from sklearn.model_selection import train_test_split
# train the model with all the features
# encode the text features
temp_data, encode = encode_text_index(data)
# normalize the numeric features
temp_data = normalize_numeric_minmax(temp_data)
# split the data into x and y
x = temp_data.drop('SalePrice', axis=1)
y = temp_data['SalePrice']
# set the first 1000 rows as train data and the rest as test data
x_test = x[1000:]
y_test = y[1000:]
x = x[:1000]
y = y[:1000]
# split the data into train and test
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0)
# train the model
model = linear_regression(x_train, y_train)
# check the model
temp_rmse = rmse(model, x_val, y_val)
print("RMSE = {}".format(round(temp_rmse, 3)))

# store the columns name
data_cols = data.columns



RMSE = 40946.091


In [9]:
# start looping through the features
for col in data_cols:
    # skip the target column
    if col == 'SalePrice':
        continue
    # remove the feature
    temp_data = data.drop(col, axis=1)
    # encode the text features
    temp_data, encode = encode_text_index(data)
    # normalize the numeric features
    temp_data = normalize_numeric_minmax(temp_data)
    # split the data into x and y
    x = temp_data.drop('SalePrice', axis=1)
    y = temp_data['SalePrice']
    # set the first 1000 rows as train data and the rest as test data
    x = x[:1000]
    y = y[:1000]
    # split the data into train and test
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0)
    # train the model
    model = linear_regression(x_train, y_train)
    # check the model
    rmse_loop = rmse(model, x_val, y_val)
    # if the new MSE is lower, remove it
    if rmse_loop < temp_rmse:
        temp_rmse = rmse_loop
        data = data.drop(col, axis=1)
    
# keep the best model
best_model = model
best_features = x.columns

In [10]:
# print the best features
print(best_features)
# print the best MSE
print("RMSE = {}".format(round(temp_rmse, 3)))

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')
RMSE = 40946.091


In [11]:
# Input test data
test_data = pd.read_csv('test.csv')
# drop the columns that are not in the best features
for col in x_test.columns:
    if col not in best_features:
        if col != 'Id':
            x_test = x_test.drop(col, axis=1)
# drop the columns with nan values
for cols in x_test.columns:
    if x_test[cols].isna().sum() > 0:
        x_test = x_test.drop(cols, axis=1)
# encode the text features
x_test, encode = encode_text_index(x_test)
# normalize the numeric features
x_test = normalize_numeric_minmax(x_test)
# do the prediction
predicted = best_model.predict(x_test)
# merge the prediction with the id
predicted = pd.DataFrame(predicted, columns=['SalePrice'])
predicted = pd.concat([test_data['Id'], predicted], axis=1)
# output the prediction with the id and save it to a csv file
predicted.to_csv('predicted.csv', index=False)
# check the RMSE
print("RMSE = {}".format(round(rmse(best_model, x_test, y_test), 3)))


        


RMSE = 49148.428
